In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
# import utils

from siuba import *
import pandas as pd

import datetime as dt
import time

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Quick example of finding cancelled service in service alerts

In [59]:
alert_query = '''SELECT * FROM `cal-itp-data-infra.staging.stg_rt__service_alerts`
WHERE date BETWEEN '2022-05-11' AND '2022-05-12' AND calitp_itp_id in (300)'''

In [60]:
alerts = query_sql(alert_query)

In [61]:
alerts.effect.unique()

array(['STOP_MOVED', 'NO_SERVICE', 'DETOUR'], dtype=object)

In [62]:
alerts>>group_by(_.calitp_itp_id)>>summarize(n = _.calitp_url_number.unique())

,calitp_itp_id,n
0,300,0


In [63]:
alerts>>filter(_.effect=="NO_SERVICE")

,calitp_itp_id,calitp_url_number,original_file_path,date,id,active_periods,informed_entities,cause,effect,url_translations,header_text_translations,description_text_translations,tts_header_text_translations,tts_description_text_translations,severity_level,key
8633,300,0,gtfs-data/rt/2022-05-11T08:00:11/300/0/gtfs_rt...,2022-05-11,ad74d4ee-87d8-4567-afc7-93a3e8f2aee5,"[{'start': 1648836000, 'end': 1660460400}]","[{'agencyId': None, 'routeId': '3501', 'routeT...",OTHER_CAUSE,NO_SERVICE,"[{'text': '', 'language': None}]",[],[],[],[],None,87ffeca7cc0c07f45df11f08daf3cf14
8634,300,0,gtfs-data/rt/2022-05-11T08:00:31/300/0/gtfs_rt...,2022-05-11,ad74d4ee-87d8-4567-afc7-93a3e8f2aee5,"[{'start': 1648836000, 'end': 1660460400}]","[{'agencyId': None, 'routeId': '3501', 'routeT...",OTHER_CAUSE,NO_SERVICE,"[{'text': '', 'language': None}]",[],[],[],[],None,1bbff8285eea75d4e157652c04c0cb1e
8635,300,0,gtfs-data/rt/2022-05-11T08:00:51/300/0/gtfs_rt...,2022-05-11,ad74d4ee-87d8-4567-afc7-93a3e8f2aee5,"[{'start': 1648836000, 'end': 1660460400}]","[{'agencyId': None, 'routeId': '3501', 'routeT...",OTHER_CAUSE,NO_SERVICE,"[{'text': '', 'language': None}]",[],[],[],[],None,428b60441122c4eb96252bffcfd14335
8636,300,0,gtfs-data/rt/2022-05-11T08:01:11/300/0/gtfs_rt...,2022-05-11,ad74d4ee-87d8-4567-afc7-93a3e8f2aee5,"[{'start': 1648836000, 'end': 1660460400}]","[{'agencyId': None, 'routeId': '3501', 'routeT...",OTHER_CAUSE,NO_SERVICE,"[{'text': '', 'language': None}]",[],[],[],[],None,0059a86ed4a748201a6d20af63da9d38
8637,300,0,gtfs-data/rt/2022-05-11T08:01:31/300/0/gtfs_rt...,2022-05-11,ad74d4ee-87d8-4567-afc7-93a3e8f2aee5,"[{'start': 1648836000, 'end': 1660460400}]","[{'agencyId': None, 'routeId': '3501', 'routeT...",OTHER_CAUSE,NO_SERVICE,"[{'text': '', 'language': None}]",[],[],[],[],None,b7c18750ba48b838186ee32a2a349238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34527,300,0,gtfs-data/rt/2022-05-12T08:58:23/300/0/gtfs_rt...,2022-05-12,30856f5c-c89a-44bf-a9f7-66f6212d6d7c,"[{'start': 1641801600, 'end': None}]","[{'agencyId': None, 'routeId': '3371', 'routeT...",OTHER_CAUSE,NO_SERVICE,[{'text': 'https://www.bigbluebus.com/servicea...,[],[],[],[],None,7dbc609025db7ef79bf5bfb2c1032554
34528,300,0,gtfs-data/rt/2022-05-12T08:58:43/300/0/gtfs_rt...,2022-05-12,30856f5c-c89a-44bf-a9f7-66f6212d6d7c,"[{'start': 1641801600, 'end': None}]","[{'agencyId': None, 'routeId': '3371', 'routeT...",OTHER_CAUSE,NO_SERVICE,[{'text': 'https://www.bigbluebus.com/servicea...,[],[],[],[],None,762ebea8a819a22f6389c3909a86a618
34529,300,0,gtfs-data/rt/2022-05-12T08:59:03/300/0/gtfs_rt...,2022-05-12,30856f5c-c89a-44bf-a9f7-66f6212d6d7c,"[{'start': 1641801600, 'end': None}]","[{'agencyId': None, 'routeId': '3371', 'routeT...",OTHER_CAUSE,NO_SERVICE,[{'text': 'https://www.bigbluebus.com/servicea...,[],[],[],[],None,91a2f598201797c5214748b0010dca54
34530,300,0,gtfs-data/rt/2022-05-12T08:59:23/300/0/gtfs_rt...,2022-05-12,30856f5c-c89a-44bf-a9f7-66f6212d6d7c,"[{'start': 1641801600, 'end': None}]","[{'agencyId': None, 'routeId': '3371', 'routeT...",OTHER_CAUSE,NO_SERVICE,[{'text': 'https://www.bigbluebus.com/servicea...,[],[],[],[],None,5deecb362743e69ca698d682eccae049


In [74]:
def get_no_service_alerts(df, analysis_date, itp_id, return_df):
    
    df = df>>filter(_.calitp_itp_id == itp_id) >> distinct(_.id, _keep_all=True)
    no_service = df>>filter(_.effect == 'NO_SERVICE')

    #get route_ids where if it appears in no_service
    no_service_rts = [entity[0]['routeId'] for entity in no_service.informed_entities.to_list()]
    
    routes = shared_utils.gtfs_utils.get_route_info(analysis_date, [itp_id])
    routes = routes >> filter(_.route_id.isin(no_service_rts))
    
    trips = shared_utils.gtfs_utils.get_trips(analysis_date, [itp_id])
    trips = trips >> filter(_.route_id.isin(no_service_rts))
    
    if return_df == 'trips':
        return trips
    elif return_df=='routes':
        return routes

In [77]:
no_service = get_no_service_alerts(alerts,'2022-05-12', 300, 'trips')

In [105]:
no_service_agg = no_service>>count(_.route_id)
no_service_agg = no_service_agg.rename(columns={'n':'n_no_service'})

In [106]:
no_service_agg

,route_id,n_no_service
0,3501,104


In [87]:
# Read in existing table

In [85]:
import utils

In [86]:
rt_sched = utils.read_data()

In [89]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
1988,290,SamTrans,1,49,49,2022-06-17,2021-08-01,2022-06-24,2,2,1.00,Friday,June,04 - Oakland


In [94]:
rt_sched = rt_sched>>filter(_.calitp_itp_id==300, _.service_date == '2022-05-12')

In [101]:
route = rt_sched>>filter(_.route_id == '3501')
route

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
1558,300,Big Blue Bus,0,3501,44,2022-05-12,2022-03-08,2022-09-01,104,0,0.00,Thursday,May,07 - Los Angeles


In [108]:
pd.merge(rt_sched, no_service_agg, how='left', on= 'route_id')

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district,n_no_service
0,300,Big Blue Bus,0,3479,1,2022-05-12,2022-03-08,2022-09-01,180,180,1.00,Thursday,May,07 - Los Angeles,NaN
1,300,Big Blue Bus,0,3480,2,2022-05-12,2022-03-08,2022-09-01,82,79,0.96,Thursday,May,07 - Los Angeles,NaN
2,300,Big Blue Bus,0,3481,3,2022-05-12,2022-03-08,2022-09-01,145,133,0.92,Thursday,May,07 - Los Angeles,NaN
3,300,Big Blue Bus,0,3485,7,2022-05-12,2022-03-08,2022-09-01,153,147,0.96,Thursday,May,07 - Los Angeles,NaN
4,300,Big Blue Bus,0,3486,8,2022-05-12,2022-03-08,2022-09-01,71,71,1.00,Thursday,May,07 - Los Angeles,NaN
5,300,Big Blue Bus,0,3487,9,2022-05-12,2022-03-08,2022-09-01,68,67,0.99,Thursday,May,07 - Los Angeles,NaN
6,300,Big Blue Bus,0,3489,R12,2022-05-12,2022-03-08,2022-09-01,153,153,1.00,Thursday,May,07 - Los Angeles,NaN
7,300,Big Blue Bus,0,3490,14,2022-05-12,2022-03-08,2022-09-01,107,107,1.00,Thursday,May,07 - Los Angeles,NaN
8,300,Big Blue Bus,0,3493,17,2022-05-12,2022-03-08,2022-09-01,88,81,0.92,Thursday,May,07 - Los Angeles,NaN
9,300,Big Blue Bus,0,3494,18,2022-05-12,2022-03-08,2022-09-01,53,53,1.00,Thursday,May,07 - Los Angeles,NaN
